In [ ]:
import numpy as np
import pandas as pd  # data processing, CSV file I/O (e.g. pd.read_csv)
URM_path="../data/interactions_and_impressions.csv"
URM_path2="../data/interactions_and_impressions_v4.csv"
URM_all_dataframe = pd.read_csv(URM_path)

URM_all_dataframe.columns = ["UserID", "ItemID", "others","Data"]
URM_all_dataframe=URM_all_dataframe.drop("others",axis=1)
URM_all_dataframe["Data"] = URM_all_dataframe["Data"].replace({0: 1, 1: 0})


URM_all_dataframe2 = pd.read_csv(URM_path2)
URM_all_dataframe2.columns = ["UserID", "ItemID","Data"]
URM_all_dataframe2 = URM_all_dataframe2[URM_all_dataframe2['Data'] != 0.01]

URM_all_dataframe2 = URM_all_dataframe2.sort_values(by=['UserID', 'ItemID', 'Data'])
URM_all_dataframe = URM_all_dataframe.sort_values(by=['UserID', 'ItemID', 'Data'])

all_users_items=list(set(list(zip(URM_all_dataframe['UserID'],URM_all_dataframe['ItemID']))))
np.random.shuffle(all_users_items)


import random
train_percentage=0.8
user_for_training=random.sample(all_users_items, round(len(all_users_items)*train_percentage))
user_for_validation=list(set(all_users_items)-set(user_for_training))
user_for_training=pd.DataFrame(user_for_training,columns=["UserID","ItemID"])
user_for_validation=pd.DataFrame(user_for_validation,columns=["UserID","ItemID"])
data_train2 = URM_all_dataframe2[URM_all_dataframe2.set_index(["UserID","ItemID"]).index.isin(user_for_training.set_index(["UserID","ItemID"]).index)]
data_train = URM_all_dataframe[URM_all_dataframe.set_index(["UserID","ItemID"]).index.isin(user_for_training.set_index(["UserID","ItemID"]).index)]
#https://stackoverflow.com/questions/54006298/select-rows-of-a-dataframe-based-on-another-dataframe-in-python



data_valid2 = URM_all_dataframe2[URM_all_dataframe2.set_index(["UserID","ItemID"]).index.isin(user_for_validation.set_index(["UserID","ItemID"]).index)]
data_valid = URM_all_dataframe[URM_all_dataframe.set_index(["UserID","ItemID"]).index.isin(user_for_validation.set_index(["UserID","ItemID"]).index)]

import numpy as np
import scipy.sparse as sps


URM_train = sps.csr_matrix((data_train["Data"].values,
                          (data_train["UserID"].values, data_train["ItemID"].values)))
URM_train2 = sps.csr_matrix((data_train2["Data"].values,
                          (data_train2["UserID"].values, data_train2["ItemID"].values)))
URM_valid = sps.csr_matrix((data_valid["Data"].values,
                             (data_valid["UserID"].values, data_valid["ItemID"].values)))
URM_valid2 = sps.csr_matrix((data_valid2["Data"].values,
                             (data_valid2["UserID"].values, data_valid2["ItemID"].values)))
URM_train,URM_train2 ,URM_valid,URM_valid2

In [72]:
data_train2

,UserID,ItemID,Data
11,0,11,0.0
25,0,21,0.0
26,0,21,0.0
27,0,21,0.0
28,0,21,0.0
...,...,...,...
8508330,41628,15181,0.0
8508331,41628,15971,1.0
8508333,41628,20448,1.0
8508334,41628,20896,0.0


In [73]:
data_train

,UserID,ItemID,Data
0,0,11,0
4,0,21,0
5,0,21,0
6,0,21,0
7,0,21,0
...,...,...,...
5826498,41628,15181,0
5826499,41628,15971,1
5826501,41628,20448,1
5826502,41628,20896,0


In [74]:

data_train.equals(data_train2)

False

In [75]:
len(data_train)

4678878

In [76]:
len(data_valid2)


1147628

In [77]:
from Utils.Evaluator import EvaluatorHoldout

evaluator_validation=EvaluatorHoldout(URM_valid,cutoff_list=[10])
evaluator_validation2=EvaluatorHoldout(URM_valid2,cutoff_list=[10])


EvaluatorHoldout: Ignoring 1615 ( 3.9%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 1615 ( 3.9%) Users that have less than 1 test interactions


In [81]:
#1
# BEST MODEL UNTIL NOW _ PROVA
from Recommenders.Hybrid.ItemUserHybridKNNRecommender import ItemUserHybridKNNRecommender

recommender_ItemKNNCFCBF = ItemUserHybridKNNRecommender(URM_train2)
recommender_ItemKNNCFCBF.fit()

result_df, _ = evaluator_validation.evaluateRecommender(recommender_ItemKNNCFCBF)
result_df

Similarity column 24507 (100.0%), 4014.61 column/sec. Elapsed time 6.10 sec
Similarity column 41629 (100.0%), 2098.02 column/sec. Elapsed time 19.84 sec
[[-3.3812413e-01 -3.3812413e-01 -3.3812413e-01 ... -3.3812413e-01
  -3.3812413e-01 -3.3812413e-01]
 [ 6.6811740e-01  3.3148074e-01  5.5787104e-01 ... -1.8927951e-01
  -2.4640305e-01 -2.8630969e-01]
 [-1.8271339e-01 -6.1182707e-04 -7.4298657e-02 ... -5.3197846e-02
  -3.0858532e-01 -1.6527626e-01]
 ...
 [ 3.1283042e-01  6.3106728e-01  3.9041653e-01 ... -2.7320513e-01
  -4.7278285e-02 -1.3880210e-01]
 [-3.3812413e-01 -2.9978958e-01 -3.3812413e-01 ... -3.3812413e-01
  -3.3812413e-01 -3.3812413e-01]
 [-3.3812413e-01 -3.3812413e-01 -3.3812413e-01 ... -3.3812413e-01
  -3.3812413e-01 -3.3812413e-01]]
[[-0.04699878 -0.04699878 -0.04699878 ... -0.04699878 -0.04699878
  -0.04699878]
 [-0.04139258 -0.04699878  0.03378833 ... -0.04699878 -0.04449808
  -0.04699878]
 [-0.01624591 -0.04699878 -0.04699878 ... -0.04699878 -0.04699878
  -0.04699878]
 ...

/Users/fasa/PycharmProjects/RecSys-Challenge-2022-Polimi/Utils/metrics.py:276: RuntimeWarning: overflow encountered in power
  return np.sum(np.divide(np.power(2, scores) - 1, np.log2(np.arange(scores.shape[0], dtype=np.float64) + 2)),


[[ 0.9331575   0.87662065  0.19130878 ...  0.16595186 -0.3196727
  -0.39125064]
 [-0.39125064 -0.2459339  -0.39125064 ... -0.39125064 -0.39125064
  -0.39125064]
 [-0.39125064 -0.11686814 -0.39125064 ... -0.39125064 -0.39125064
  -0.39125064]
 ...
 [-0.39125064 -0.39125064 -0.39125064 ... -0.39125064 -0.39125064
  -0.39125064]
 [ 0.35039994  0.00295679 -0.20891866 ... -0.37971732 -0.30198094
  -0.36922416]
 [ 0.389416    0.29367247 -0.09832957 ...  0.06022141 -0.06101435
  -0.34313107]]
[[-0.02475367 -0.02964734 -0.04512678 ... -0.03662778 -0.04644375
  -0.04644375]
 [-0.04644375 -0.03957249 -0.04644375 ... -0.04644375 -0.04644375
  -0.04644375]
 [-0.04644375 -0.04644375 -0.04644375 ... -0.04644375 -0.04644375
  -0.04644375]
 ...
 [-0.04644375 -0.04644375 -0.04644375 ... -0.04644375 -0.04644375
  -0.04644375]
 [-0.04644375 -0.02966615 -0.04278756 ... -0.04644375 -0.04644375
  -0.04644375]
 [-0.02458727 -0.00460298 -0.04644375 ... -0.02959694 -0.03582711
  -0.04644375]]
[[-0.25394312 -0.

,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,MRR,NDCG,F1,HIT_RATE,ARHR_ALL_HITS,...,COVERAGE_USER,COVERAGE_USER_HIT,USERS_IN_GT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
cutoff,,,,,,,,,,,,,,,,,,,,,
10,0.050945,0.088729,0.083468,0.025971,0.044123,0.168771,inf,0.063271,0.329685,0.207422,...,0.961205,0.316894,0.961205,0.039123,9.160113,0.992365,0.07705,0.678863,4.229822,0.264116


In [ ]:
#1
# BEST MODEL UNTIL NOW - CONTROPROVA
from Recommenders.Hybrid.ItemUserHybridKNNRecommender import ItemUserHybridKNNRecommender

recommender_ItemKNNCFCBF = ItemUserHybridKNNRecommender(URM_train)
recommender_ItemKNNCFCBF.fit()

result_df, _ = evaluator_validation.evaluateRecommender(recommender_ItemKNNCFCBF)
result_df

Similarity column 24507 (100.0%), 4842.97 column/sec. Elapsed time 5.06 sec
Similarity column 41629 (100.0%), 2192.84 column/sec. Elapsed time 18.98 sec
[[-3.3812413e-01 -3.3812413e-01 -3.3812413e-01 ... -3.3812413e-01
  -3.3812413e-01 -3.3812413e-01]
 [ 6.6811740e-01  3.3148074e-01  5.5787104e-01 ... -1.8927951e-01
  -2.4640305e-01 -2.8630969e-01]
 [-1.8271339e-01 -6.1182707e-04 -7.4298657e-02 ... -5.3197846e-02
  -3.0858532e-01 -1.6527626e-01]
 ...
 [ 3.1283042e-01  6.3106728e-01  3.9041653e-01 ... -2.7320513e-01
  -4.7278285e-02 -1.3880210e-01]
 [-3.3812413e-01 -2.9978958e-01 -3.3812413e-01 ... -3.3812413e-01
  -3.3812413e-01 -3.3812413e-01]
 [-3.3812413e-01 -3.3812413e-01 -3.3812413e-01 ... -3.3812413e-01
  -3.3812413e-01 -3.3812413e-01]]
[[-0.04699878 -0.04699878 -0.04699878 ... -0.04699878 -0.04699878
  -0.04699878]
 [-0.04139258 -0.04699878  0.03378833 ... -0.04699878 -0.04449808
  -0.04699878]
 [-0.01624591 -0.04699878 -0.04699878 ... -0.04699878 -0.04699878
  -0.04699878]
 ...

/Users/fasa/PycharmProjects/RecSys-Challenge-2022-Polimi/Utils/metrics.py:276: RuntimeWarning: overflow encountered in power
  return np.sum(np.divide(np.power(2, scores) - 1, np.log2(np.arange(scores.shape[0], dtype=np.float64) + 2)),


[[ 0.9331575   0.87662065  0.19130878 ...  0.16595186 -0.3196727
  -0.39125064]
 [-0.39125064 -0.2459339  -0.39125064 ... -0.39125064 -0.39125064
  -0.39125064]
 [-0.39125064 -0.11686814 -0.39125064 ... -0.39125064 -0.39125064
  -0.39125064]
 ...
 [-0.39125064 -0.39125064 -0.39125064 ... -0.39125064 -0.39125064
  -0.39125064]
 [ 0.35039994  0.00295679 -0.20891866 ... -0.37971732 -0.30198094
  -0.36922416]
 [ 0.389416    0.29367247 -0.09832957 ...  0.06022141 -0.06101435
  -0.34313107]]
[[-0.02475367 -0.02964734 -0.04512678 ... -0.03662778 -0.04644375
  -0.04644375]
 [-0.04644375 -0.03957249 -0.04644375 ... -0.04644375 -0.04644375
  -0.04644375]
 [-0.04644375 -0.04644375 -0.04644375 ... -0.04644375 -0.04644375
  -0.04644375]
 ...
 [-0.04644375 -0.04644375 -0.04644375 ... -0.04644375 -0.04644375
  -0.04644375]
 [-0.04644375 -0.02966615 -0.04278756 ... -0.04644375 -0.04644375
  -0.04644375]
 [-0.02458727 -0.00460298 -0.04644375 ... -0.02959694 -0.03582711
  -0.04644375]]
[[-0.25394312 -0.

In [80]:
#1
# BEST MODEL UNTIL NOW _ PROVA
from Recommenders.Hybrid.ItemUserHybridKNNRecommender import ItemUserHybridKNNRecommender

recommender_ItemKNNCFCBF = ItemUserHybridKNNRecommender(URM_train2)
recommender_ItemKNNCFCBF.fit()

result_df, _ = evaluator_validation2.evaluateRecommender(recommender_ItemKNNCFCBF)
result_df

Similarity column 24507 (100.0%), 5522.93 column/sec. Elapsed time 4.44 sec
Similarity column 41629 (100.0%), 1581.75 column/sec. Elapsed time 26.32 sec
[[-3.3812413e-01 -3.3812413e-01 -3.3812413e-01 ... -3.3812413e-01
  -3.3812413e-01 -3.3812413e-01]
 [ 6.6811740e-01  3.3148074e-01  5.5787104e-01 ... -1.8927951e-01
  -2.4640305e-01 -2.8630969e-01]
 [-1.8271339e-01 -6.1182707e-04 -7.4298657e-02 ... -5.3197846e-02
  -3.0858532e-01 -1.6527626e-01]
 ...
 [ 3.1283042e-01  6.3106728e-01  3.9041653e-01 ... -2.7320513e-01
  -4.7278285e-02 -1.3880210e-01]
 [-3.3812413e-01 -2.9978958e-01 -3.3812413e-01 ... -3.3812413e-01
  -3.3812413e-01 -3.3812413e-01]
 [-3.3812413e-01 -3.3812413e-01 -3.3812413e-01 ... -3.3812413e-01
  -3.3812413e-01 -3.3812413e-01]]
[[-0.04699878 -0.04699878 -0.04699878 ... -0.04699878 -0.04699878
  -0.04699878]
 [-0.04139258 -0.04699878  0.03378833 ... -0.04699878 -0.04449808
  -0.04699878]
 [-0.01624591 -0.04699878 -0.04699878 ... -0.04699878 -0.04699878
  -0.04699878]
 ...

/Users/fasa/PycharmProjects/RecSys-Challenge-2022-Polimi/Utils/metrics.py:276: RuntimeWarning: overflow encountered in power
  return np.sum(np.divide(np.power(2, scores) - 1, np.log2(np.arange(scores.shape[0], dtype=np.float64) + 2)),


[[ 0.3172215   0.1213162   0.41985998 ... -0.40589383  0.26864055
  -0.28252238]
 [-0.40589383 -0.40589383 -0.40589383 ... -0.40589383 -0.40589383
  -0.40589383]
 [-0.40589383 -0.40589383 -0.32330316 ... -0.40589383 -0.40589383
  -0.40589383]
 ...
 [ 0.12350119  0.05493488  0.05624962 ... -0.10737585  0.3537746
  -0.0615656 ]
 [ 0.10752659 -0.3070806  -0.37597302 ... -0.40589383 -0.40589383
  -0.40589383]
 [-0.0381002  -0.13554159 -0.18681686 ... -0.40589383 -0.34581923
   0.34516817]]
[[-0.04691719 -0.04691719 -0.04090076 ... -0.04691719 -0.04062768
  -0.04691719]
 [-0.04691719 -0.04691719 -0.04691719 ... -0.04691719 -0.04691719
  -0.04691719]
 [-0.04691719 -0.04691719 -0.03268892 ... -0.04691719 -0.04691719
  -0.04691719]
 ...
 [-0.04032902 -0.02769356 -0.04691719 ... -0.026366    0.01702584
  -0.03584449]
 [-0.03376542 -0.04559053 -0.04691719 ... -0.04691719 -0.04691719
  -0.04691719]
 [-0.00530716 -0.00791192  0.00101664 ... -0.04691719 -0.04691719
   0.01328242]]


/Users/fasa/PycharmProjects/RecSys-Challenge-2022-Polimi/Utils/metrics.py:270: RuntimeWarning: invalid value encountered in double_scalars
  ndcg_ = rank_dcg / ideal_dcg


[[ 0.9331575   0.87662065  0.19130878 ...  0.16595186 -0.3196727
  -0.39125064]
 [-0.39125064 -0.2459339  -0.39125064 ... -0.39125064 -0.39125064
  -0.39125064]
 [-0.39125064 -0.11686814 -0.39125064 ... -0.39125064 -0.39125064
  -0.39125064]
 ...
 [-0.39125064 -0.39125064 -0.39125064 ... -0.39125064 -0.39125064
  -0.39125064]
 [ 0.35039994  0.00295679 -0.20891866 ... -0.37971732 -0.30198094
  -0.36922416]
 [ 0.389416    0.29367247 -0.09832957 ...  0.06022141 -0.06101435
  -0.34313107]]
[[-0.02475367 -0.02964734 -0.04512678 ... -0.03662778 -0.04644375
  -0.04644375]
 [-0.04644375 -0.03957249 -0.04644375 ... -0.04644375 -0.04644375
  -0.04644375]
 [-0.04644375 -0.04644375 -0.04644375 ... -0.04644375 -0.04644375
  -0.04644375]
 ...
 [-0.04644375 -0.04644375 -0.04644375 ... -0.04644375 -0.04644375
  -0.04644375]
 [-0.04644375 -0.02966615 -0.04278756 ... -0.04644375 -0.04644375
  -0.04644375]
 [-0.02458727 -0.00460298 -0.04644375 ... -0.02959694 -0.03582711
  -0.04644375]]
[[-0.25394312 -0.

,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,MRR,NDCG,F1,HIT_RATE,ARHR_ALL_HITS,...,COVERAGE_USER,COVERAGE_USER_HIT,USERS_IN_GT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
cutoff,,,,,,,,,,,,,,,,,,,,,
10,0.050945,0.088729,0.083468,0.025971,0.044123,0.168771,NaN,0.063271,0.329685,0.207422,...,0.961205,0.316894,0.961205,0.039123,9.160113,0.992365,0.07705,0.678863,4.229822,0.264116
